In [1]:
#tải các thư viện cần thiết
!pip install transformer
!pip install seqeval

ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=52e4a3108388eab9038ffdece3c60cf8e87236c00c88837028ae2d2bc8e4252c
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
#import các thư viện cần thiết

import numpy as np
import pandas as pd
import torch

#Embedding
from transformers import AutoModel, AutoTokenizer, AutoModelForTokenClassification

#Modeling
from torch.utils.data import DataLoader
from torch.optim import SGD, Adam
from seqeval.metrics import classification_report
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [4]:
#hàm đọc dữ liệu
def read_dataset(file_path):
    tokens=[]
    ner_tags=[]
    ids=[]
    count=1
    with open(file_path) as f:
        lines=f.readlines()
        ts=[]
        nts=[]
        for line in lines:
            line = line.split()
            if len(line)==0:
                ids.append(count)
                tokens.append(ts)
                ner_tags.append(nts)
                ts=[]
                nts=[]
                count+=1
            else:
                ts.append(line[0])
                nts.append(line[-1])
    data = pd.DataFrame({'Id':ids, 'NER_tags':ner_tags, 'Tokens':tokens})
    return data

In [5]:
def get_NER_labels(data):
    NERs = list(data['NER_tags'].values)
    labels_list = []
    for value in NERs:
        labels_list = labels_list + value
    types = list(set(labels_list))
    return types

In [6]:
train_df = read_dataset('/kaggle/input/covid19vi/word/train_word.conll')
val_df = read_dataset('/kaggle/input/covid19vi/word/dev_word.conll')

In [7]:
train_df.head()

,Id,NER_tags,Tokens
0,1,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Đồng_thời, ,, bệnh_viện, tiếp_tục, thực_hiện,..."
1,2,"[O, O, O, O, O, O, O, B-SYMPTOM_AND_DISEASE, I...","["", Số, bệnh_viện, có_thể, tiếp_nhận, bệnh_nhâ..."
2,3,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Ngoài_ra, ,, những, người, tiếp_xúc, gián_tiế..."
3,4,"[O, O, O, O, O, O, B-LOCATION, O, B-LOCATION, ...","[Bà, này, khi, trở, về, quá_cảnh, Doha, (, Qat..."
4,5,"[O, O, B-PATIENT_ID, O, O, O, O, O, O, B-PATIE...","["", Bệnh_nhân, 523, "", và, chồng, là, "", bệnh_..."


In [8]:
unique_labels = get_NER_labels(train_df)

# **TẠO LỚP DATASET BẰNG PYTORCH DATASET**

In [9]:
def align_label(text, labels, flag=False):
    label_all_tokens = flag #flag xác định cách thực hiện align_label
    
    tokenized_input = tokenizer(text, padding='max_length', max_length=256, truncation=True, is_split_into_words=True)
    word_ids = tokenized_input.input_ids
    
    start_part = True
    label_ids = []
    count = 0
    
    for i in range(len(word_ids)):
        
        if word_ids[i] == 0 or word_ids[i] == 1 or word_ids[i] == 2:
            label_ids.append(-100)
            
        elif count < len(text) and ''.join(tokenizer.decode(tokenized_input['input_ids'][i]).split()) == text[count]:
            label_ids.append(labels_to_ids[labels[count]])
            count+=1
            start_part = True
        else:
            if start_part:
                label_ids.append(labels_to_ids[labels[count]])
                count+=1
                start_part = False
            else:
                label_ids.append(labels_to_ids[labels[count]] if label_all_tokens else -100)
                
    return label_ids

class DataSet(torch.utils.data.Dataset):

    def __init__(self, df, flag_align_label=False):

        lb = df['NER_tags'].values.tolist()
        txt = df['Tokens'].values.tolist()
        self.texts = [tokenizer(i, padding='max_length', max_length = 256,
                                truncation=True, return_tensors="pt", is_split_into_words=True) for i in txt]
        self.labels = [align_label(i,j,flag_align_label) for i,j in zip(txt, lb)]
        
    def __len__(self):

        return len(self.labels)

    def get_data(self, idx):
        return self.texts[idx]

    def get_labels(self, idx):
        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):
        data = self.get_data(idx)
        labels = self.get_labels(idx)

        return data, labels

**Tạo 2 dictionary để xác định id nào sẽ là label nào và ngược lại.**

In [10]:
labels_to_ids = {k: v for v, k in enumerate(unique_labels)}
ids_to_labels = {v: k for v, k in enumerate(unique_labels)}

In [11]:
labels_to_ids

{'B-DATE': 0,
 'I-JOB': 1,
 'B-PATIENT_ID': 2,
 'I-LOCATION': 3,
 'O': 4,
 'I-SYMPTOM_AND_DISEASE': 5,
 'I-TRANSPORTATION': 6,
 'B-NAME': 7,
 'B-AGE': 8,
 'I-ORGANIZATION': 9,
 'B-ORGANIZATION': 10,
 'B-GENDER': 11,
 'I-NAME': 12,
 'B-TRANSPORTATION': 13,
 'I-AGE': 14,
 'I-DATE': 15,
 'B-SYMPTOM_AND_DISEASE': 16,
 'I-PATIENT_ID': 17,
 'B-LOCATION': 18,
 'B-JOB': 19}

# **BUILD MODEL PHOBERT FOR TOKEN CLASSIFICATION**

In [12]:
class PhoBertModel(torch.nn.Module):

    def __init__(self):

        super(PhoBertModel, self).__init__()

        self.phobert = AutoModelForTokenClassification.from_pretrained("vinai/phobert-base", num_labels=len(unique_labels))

    def forward(self, input_id, mask, label):
        
        output = self.phobert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)
        
        return output

# **TRAIN MODEL**

In [13]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [14]:
def train_loop(model, train_df, val_df, flag_align_label):

    train_dataset = DataSet(train_df, flag_align_label)
    val_dataset = DataSet(val_df, flag_align_label)

    train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()
        
    min_val_loss = 1000
    count = 0
    
    for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)

            for i in range(logits.shape[0]):

                logits_clean = logits[i][train_label[i] != -100]
                label_clean = train_label[i][train_label[i] != -100]

                predictions = logits_clean.argmax(dim=1)
                acc = (predictions == label_clean).float().mean()
                total_acc_train += acc
                total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].squeeze(1).to(device)
            input_id = val_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, val_label)

            for i in range(logits.shape[0]):

                logits_clean = logits[i][val_label[i] != -100]
                label_clean = val_label[i][val_label[i] != -100]

                predictions = logits_clean.argmax(dim=1)
                acc = (predictions == label_clean).float().mean()
                total_acc_val += acc
                total_loss_val += loss.item()

        val_accuracy = total_acc_val / len(val_df)
        val_loss = total_loss_val / len(val_df)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(train_df): .3f} | Accuracy: {total_acc_train / len(train_df): .3f} | Val_Loss: {total_loss_val / len(val_df): .3f} | Accuracy: {total_acc_val / len(val_df): .3f}')
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            torch.save(model.state_dict(), 'phobert_base_ner')
            count = epoch_num
        if epoch_num - count >= 5:
            return
        
LEARNING_RATE = 5e-5
EPOCHS = 30
BATCH_SIZE = 32

model = PhoBertModel()
train_loop(model, train_df, val_df, False)

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 158/158 [02:00<00:00,  1.31it/s]


Epochs: 1 | Loss:  0.495 | Accuracy:  0.898 | Val_Loss:  0.116 | Accuracy:  0.976


100%|██████████| 158/158 [02:00<00:00,  1.32it/s]


Epochs: 2 | Loss:  0.081 | Accuracy:  0.985 | Val_Loss:  0.073 | Accuracy:  0.984


...

Epochs: 7 | Loss:  0.018 | Accuracy:  0.996 | Val_Loss:  0.070 | Accuracy:  0.985


100%|██████████| 158/158 [02:00<00:00,  1.32it/s]


Epochs: 8 | Loss:  0.014 | Accuracy:  0.997 | Val_Loss:  0.075 | Accuracy:  0.985


100%|██████████| 158/158 [02:00<00:00,  1.31it/s]


Epochs: 9 | Loss:  0.011 | Accuracy:  0.998 | Val_Loss:  0.072 | Accuracy:  0.985


# **EVALUATE MODEL**

In [15]:
test_df = read_dataset('/kaggle/input/covid19vi/word/test_word.conll')

In [16]:
def evaluate(model, test_df, flag_align_label, ids_to_labels):

    test_dataset = DataSet(test_df, flag_align_label)

    test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    predictions = []
    labels = []
    
    for test_data, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_data['attention_mask'].squeeze(1).to(device)

            input_id = test_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, test_label)

            for i in range(logits.shape[0]):
                cleaned_logits = logits[i][test_label[i] != -100].argmax(dim=1)
                predictions.append([ids_to_labels[val.item()] for val in cleaned_logits])
                cleaned_labels = test_label[i][test_label[i] != -100]
                labels.append([ids_to_labels[val.item()] for val in cleaned_labels])
    print(classification_report(y_pred=predictions, y_true=labels, digits=3))

evaluate(model, test_df, False, ids_to_labels)

                     precision    recall  f1-score   support

                AGE      0.978     0.961     0.970       568
               DATE      0.981     0.990     0.985      1641
             GENDER      0.970     0.978     0.974       458
                JOB      0.773     0.846     0.808       169
           LOCATION      0.946     0.953     0.949      4346
               NAME      0.928     0.943     0.935       314
       ORGANIZATION      0.859     0.900     0.879       768
         PATIENT_ID      0.982     0.984     0.983      1982
SYMPTOM_AND_DISEASE      0.875     0.871     0.873      1135
     TRANSPORTATION      0.944     0.984     0.964       189

          micro avg      0.943     0.952     0.948     11570
          macro avg      0.924     0.941     0.932     11570
       weighted avg      0.944     0.952     0.948     11570



# **INFERENCE**

In [39]:
def align_word_ids(text, flag):
    label_all_tokens = flag
    
    text = text.split()
  
    tokenized_inputs = tokenizer(text, padding='max_length', max_length=256, truncation=True, is_split_into_words=True)

    word_ids = tokenized_inputs.input_ids

    start_part = True
    label_ids = []
    count = 0
    
    for i in range(len(word_ids)):
        
        if word_ids[i] == 0 or word_ids[i] == 1 or word_ids[i] == 2:
            label_ids.append(-100)
            
        elif count < len(text) and ''.join(tokenizer.decode(tokenized_inputs['input_ids'][i]).split()) == text[count]:
            label_ids.append(1)
            count+=1
            start_part = True
        else:
            if start_part:
                label_ids.append(1)
                count+=1
                start_part = False
            else:
                label_ids.append(1 if label_all_tokens else -100)           
    return label_ids

In [18]:
def ner(model, sentence, flag_align_label):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
    text = tokenizer(sentence, padding='max_length', max_length = 256, truncation=True, return_tensors="pt")

    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)
    label_ids = torch.Tensor(align_word_ids(sentence, flag_align_label)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    print(sentence)
    print(prediction_label)

In [40]:
ner(model,
    'Bệnh_nhân nhập_viện tối_qua ở Bệnh_Viện 115 là bệnh_nhân thứ 82 , di_chuyển qua nhiều thành_phố bằng xe biển_hiệu E-402',
    flag_align_label=False)

Bệnh_nhân nhập_viện tối_qua ở Bệnh_Viện 115 là bệnh_nhân thứ 82 , di_chuyển qua nhiều thành_phố bằng xe biển_hiệu E-402
['O', 'O', 'O', 'O', 'B-LOCATION', 'I-LOCATION', 'O', 'O', 'O', 'B-PATIENT_ID', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-TRANSPORTATION']
